In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import scipy 
import pandas as pd
import shutil

In [2]:
df=pd.read_csv(r"C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\VideoDemographics.csv")
df.head(3)

,ActorID,Age,Sex,Race,Ethnicity
0,1001,51,Male,Caucasian,Not Hispanic
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic


In [4]:
female_ids=[str(df.loc[i].iat[0]) for i in df.index if df.loc[i].iat[2]=="Female"]

In [5]:
audio_folder_name=r'C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV'
dest_folder_name= r'C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\FemaleAudioWAV'
file_names = listdir(audio_folder_name)
for f in file_names:
    
    x=f.split("_")
    if x[0] not in female_ids:
       continue
    #print(f)
    shutil.move(audio_folder_name + '\\' +f, dest_folder_name)
    

In [117]:
##FUNCTIONS

def convert_single_audio_to_image(audio_path, image_path, spectrogram_dimensions = (64,64)):
    data, sr = librosa.load(audio_path, sr = 44100)  
    print("sr={}".format(sr))
    #Make a mel spectrogram from audio
    spec = librosa.feature.melspectrogram(y=data, sr=sr)
    #Convert amplitude to decibels
    db_spec = librosa.power_to_db(spec, ref=np.max)
    fig = plt.figure()
    
    
   # fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
    
    #Display final mel spectrogram
    librosa.display.specshow(db_spec, sr=sr,x_axis='time',y_axis='mel')
    plt.colorbar()
    fig.savefig(image_path)
    plt.close(fig)


In [118]:
if __name__ == '__main__':
    audio_folder_name= dest_folder_name
    image_folder_name= r'C:\Users\pinkb\OneDrive\Documents\Spring 2020\Deep Learning\Emotion Classification\FemaleAudioMelspecs'
    file_names = listdir(audio_folder_name)
#     file_names = [f for f in listdir(audio_folder_name)]
    print("count of Wav_files in {}= {}".format(len(file_names),audio_folder_name))
    for file_name in file_names:
        name, ext = os.path.splitext(file_name)
        print(file_name)
            
        audio_path = audio_folder_name + '\\' + file_name
        spectogram_path = image_folder_name + '\\' + file_name.replace('.wav', '.png')
        convert_single_audio_to_image(audio_path,spectogram_path)
    image_files= listdir(image_folder_path)
    print("count of Wav_files in {}= {}".format(len(image_files),image_folder_path))
        

count of Wav_files in 3512= C:\Users\pinkb\OneDrive\Documents\Spring 2020\Deep Learning\Emotion Classification\FemaleAudioWavFiles
1002_DFA_ANG_XX.wav
sr=44100
1002_DFA_DIS_XX.wav
sr=44100
1002_DFA_FEA_XX.wav
sr=44100
1002_DFA_HAP_XX.wav
sr=44100
1002_DFA_NEU_XX.wav
sr=44100
1002_DFA_SAD_XX.wav
sr=44100
1002_IEO_ANG_HI.wav
sr=44100
1002_IEO_ANG_LO.wav
sr=44100
1002_IEO_ANG_MD.wav
sr=44100
1002_IEO_DIS_HI.wav
sr=44100
1002_IEO_DIS_LO.wav
sr=44100
1002_IEO_DIS_MD.wav
sr=44100
1002_IEO_FEA_HI.wav
sr=44100
1002_IEO_FEA_LO.wav
sr=44100
1002_IEO_FEA_MD.wav
sr=44100
1002_IEO_HAP_HI.wav
sr=44100
1002_IEO_HAP_LO.wav
sr=44100
1002_IEO_HAP_MD.wav
sr=44100
1002_IEO_NEU_XX.wav
sr=44100
1002_IEO_SAD_HI.wav
sr=44100
1002_IEO_SAD_LO.wav
sr=44100
1002_IEO_SAD_MD.wav
sr=44100
1002_IOM_ANG_XX.wav
sr=44100
1002_IOM_DIS_XX.wav
sr=44100
1002_IOM_FEA_XX.wav
sr=44100
1002_IOM_HAP_XX.wav
sr=44100
1002_IOM_NEU_XX.wav
sr=44100
1002_IOM_SAD_XX.wav
sr=44100
1002_ITH_ANG_XX.wav
sr=44100
1002_ITH_DIS_XX.wav
sr=44100

1006_ITH_SAD_XX.wav
sr=44100
1006_ITS_ANG_XX.wav
sr=44100
1006_ITS_DIS_XX.wav
sr=44100
1006_ITS_FEA_XX.wav
sr=44100
1006_ITS_HAP_XX.wav
sr=44100
1006_ITS_NEU_XX.wav
sr=44100
1006_ITS_SAD_XX.wav
sr=44100
1006_IWL_ANG_XX.wav
sr=44100
1006_IWL_DIS_XX.wav
sr=44100
1006_IWL_FEA_XX.wav
sr=44100
1006_IWL_HAP_XX.wav
sr=44100
1006_IWL_NEU_XX.wav
sr=44100
1006_IWL_SAD_XX.wav
sr=44100
1006_IWW_ANG_XX.wav
sr=44100
1006_IWW_DIS_XX.wav
sr=44100
1006_IWW_FEA_XX.wav
sr=44100
1006_IWW_HAP_XX.wav
sr=44100
1006_IWW_NEU_XX.wav
sr=44100
1006_IWW_SAD_XX.wav
sr=44100
1006_MTI_ANG_XX.wav
sr=44100
1006_MTI_DIS_XX.wav
sr=44100
1006_MTI_FEA_XX.wav
sr=44100
1006_MTI_HAP_XX.wav
sr=44100
1006_MTI_NEU_XX.wav
sr=44100
1006_MTI_SAD_XX.wav
sr=44100
1006_TAI_ANG_XX.wav
sr=44100
1006_TAI_DIS_XX.wav
sr=44100
1006_TAI_FEA_XX.wav
sr=44100
1006_TAI_HAP_XX.wav
sr=44100
1006_TAI_NEU_XX.wav
sr=44100
1006_TAI_SAD_XX.wav
sr=44100
1006_TIE_ANG_XX.wav
sr=44100
1006_TIE_DIS_XX.wav
sr=44100
1006_TIE_FEA_XX.wav
sr=44100
1006_TIE_HAP_X

sr=44100
1010_DFA_ANG_XX.wav
sr=44100
1010_DFA_DIS_XX.wav
sr=44100
1010_DFA_FEA_XX.wav
sr=44100
1010_DFA_HAP_XX.wav
sr=44100
1010_DFA_NEU_XX.wav
sr=44100
1010_DFA_SAD_XX.wav
sr=44100
1010_IEO_ANG_HI.wav
sr=44100
1010_IEO_ANG_LO.wav
sr=44100
1010_IEO_ANG_MD.wav
sr=44100
1010_IEO_DIS_HI.wav
sr=44100
1010_IEO_DIS_LO.wav
sr=44100
1010_IEO_DIS_MD.wav
sr=44100
1010_IEO_FEA_HI.wav
sr=44100
1010_IEO_FEA_LO.wav
sr=44100
1010_IEO_FEA_MD.wav
sr=44100
1010_IEO_HAP_HI.wav
sr=44100
1010_IEO_HAP_LO.wav
sr=44100
1010_IEO_HAP_MD.wav
sr=44100
1010_IEO_NEU_XX.wav
sr=44100
1010_IEO_SAD_HI.wav
sr=44100
1010_IEO_SAD_LO.wav
sr=44100
1010_IEO_SAD_MD.wav
sr=44100
1010_IOM_ANG_XX.wav
sr=44100
1010_IOM_DIS_XX.wav
sr=44100
1010_IOM_FEA_XX.wav
sr=44100
1010_IOM_HAP_XX.wav
sr=44100
1010_IOM_NEU_XX.wav
sr=44100
1010_IOM_SAD_XX.wav
sr=44100
1010_ITH_ANG_XX.wav
sr=44100
1010_ITH_DIS_XX.wav
sr=44100
1010_ITH_FEA_XX.wav
sr=44100
1010_ITH_HAP_XX.wav
sr=44100
1010_ITH_NEU_XX.wav
sr=44100
1010_ITH_SAD_XX.wav
sr=44100
1010_

sr=44100
1018_ITS_HAP_XX.wav
sr=44100
1018_ITS_NEU_XX.wav
sr=44100
1018_ITS_SAD_XX.wav
sr=44100
1018_IWL_ANG_XX.wav
sr=44100
1018_IWL_DIS_XX.wav
sr=44100
1018_IWL_FEA_XX.wav
sr=44100
1018_IWL_HAP_XX.wav
sr=44100
1018_IWL_NEU_XX.wav
sr=44100
1018_IWL_SAD_XX.wav
sr=44100
1018_IWW_ANG_XX.wav
sr=44100
1018_IWW_DIS_XX.wav
sr=44100
1018_IWW_FEA_XX.wav
sr=44100
1018_IWW_HAP_XX.wav
sr=44100
1018_IWW_NEU_XX.wav
sr=44100
1018_IWW_SAD_XX.wav
sr=44100
1018_MTI_ANG_XX.wav
sr=44100
1018_MTI_DIS_XX.wav
sr=44100
1018_MTI_FEA_XX.wav
sr=44100
1018_MTI_HAP_XX.wav
sr=44100
1018_MTI_NEU_XX.wav
sr=44100
1018_MTI_SAD_XX.wav
sr=44100
1018_TAI_ANG_XX.wav
sr=44100
1018_TAI_DIS_XX.wav
sr=44100
1018_TAI_FEA_XX.wav
sr=44100
1018_TAI_HAP_XX.wav
sr=44100
1018_TAI_NEU_XX.wav
sr=44100
1018_TAI_SAD_XX.wav
sr=44100
1018_TIE_ANG_XX.wav
sr=44100
1018_TIE_DIS_XX.wav
sr=44100
1018_TIE_FEA_XX.wav
sr=44100
1018_TIE_HAP_XX.wav
sr=44100
1018_TIE_NEU_XX.wav
sr=44100
1018_TIE_SAD_XX.wav
sr=44100
1018_TSI_ANG_XX.wav
sr=44100
1018_

sr=44100
1024_TSI_NEU_XX.wav
sr=44100
1024_TSI_SAD_XX.wav
sr=44100
1024_WSI_ANG_XX.wav
sr=44100
1024_WSI_DIS_XX.wav
sr=44100
1024_WSI_FEA_XX.wav
sr=44100
1024_WSI_HAP_XX.wav
sr=44100
1024_WSI_NEU_XX.wav
sr=44100
1024_WSI_SAD_XX.wav
sr=44100
1025_DFA_ANG_XX.wav
sr=44100
1025_DFA_DIS_XX.wav
sr=44100
1025_DFA_FEA_XX.wav
sr=44100
1025_DFA_HAP_XX.wav
sr=44100
1025_DFA_NEU_XX.wav
sr=44100
1025_DFA_SAD_XX.wav
sr=44100
1025_IEO_ANG_HI.wav
sr=44100
1025_IEO_ANG_LO.wav
sr=44100
1025_IEO_ANG_MD.wav
sr=44100
1025_IEO_DIS_HI.wav
sr=44100
1025_IEO_DIS_LO.wav
sr=44100
1025_IEO_DIS_MD.wav
sr=44100
1025_IEO_FEA_HI.wav
sr=44100
1025_IEO_FEA_LO.wav
sr=44100
1025_IEO_FEA_MD.wav
sr=44100
1025_IEO_HAP_HI.wav
sr=44100
1025_IEO_HAP_LO.wav
sr=44100
1025_IEO_HAP_MD.wav
sr=44100
1025_IEO_NEU_XX.wav
sr=44100
1025_IEO_SAD_HI.wav
sr=44100
1025_IEO_SAD_LO.wav
sr=44100
1025_IEO_SAD_MD.wav
sr=44100
1025_IOM_ANG_XX.wav
sr=44100
1025_IOM_DIS_XX.wav
sr=44100
1025_IOM_FEA_XX.wav
sr=44100
1025_IOM_HAP_XX.wav
sr=44100
1025_

...

sr=44100
1063_IEO_FEA_LO.wav
sr=44100
1063_IEO_FEA_MD.wav
sr=44100
1063_IEO_HAP_HI.wav
sr=44100
1063_IEO_HAP_LO.wav
sr=44100
1063_IEO_HAP_MD.wav
sr=44100
1063_IEO_NEU_XX.wav
sr=44100
1063_IEO_SAD_HI.wav
sr=44100
1063_IEO_SAD_LO.wav
sr=44100
1063_IEO_SAD_MD.wav
sr=44100
1063_IOM_ANG_XX.wav
sr=44100
1063_IOM_DIS_XX.wav
sr=44100
1063_IOM_FEA_XX.wav
sr=44100
1063_IOM_HAP_XX.wav
sr=44100
1063_IOM_NEU_XX.wav
sr=44100
1063_IOM_SAD_XX.wav
sr=44100
1063_ITH_ANG_XX.wav
sr=44100
1063_ITH_DIS_XX.wav
sr=44100
1063_ITH_FEA_XX.wav
sr=44100
1063_ITH_HAP_XX.wav
sr=44100
1063_ITH_NEU_XX.wav
sr=44100
1063_ITH_SAD_XX.wav
sr=44100
1063_ITS_ANG_XX.wav
sr=44100
1063_ITS_DIS_XX.wav
sr=44100
1063_ITS_FEA_XX.wav
sr=44100
1063_ITS_HAP_XX.wav
sr=44100
1063_ITS_NEU_XX.wav
sr=44100
1063_ITS_SAD_XX.wav
sr=44100
1063_IWL_ANG_XX.wav
sr=44100
1063_IWL_DIS_XX.wav
sr=44100
1063_IWL_FEA_XX.wav
sr=44100
1063_IWL_HAP_XX.wav
sr=44100
1063_IWL_NEU_XX.wav
sr=44100
1063_IWL_SAD_XX.wav
sr=44100
1063_IWW_ANG_XX.wav
sr=44100
1063_

sr=44100
1074_IWW_NEU_XX.wav
sr=44100
1074_IWW_SAD_XX.wav
sr=44100
1074_MTI_ANG_XX.wav
sr=44100
1074_MTI_DIS_XX.wav
sr=44100
1074_MTI_FEA_XX.wav
sr=44100
1074_MTI_HAP_XX.wav
sr=44100
1074_MTI_NEU_XX.wav
sr=44100
1074_MTI_SAD_XX.wav
sr=44100
1074_TAI_ANG_XX.wav
sr=44100
1074_TAI_DIS_XX.wav
sr=44100
1074_TAI_FEA_XX.wav
sr=44100
1074_TAI_HAP_XX.wav
sr=44100
1074_TAI_NEU_XX.wav
sr=44100
1074_TAI_SAD_XX.wav
sr=44100
1074_TIE_ANG_XX.wav
sr=44100
1074_TIE_DIS_XX.wav
sr=44100
1074_TIE_FEA_XX.wav
sr=44100
1074_TIE_HAP_XX.wav
sr=44100
1074_TIE_NEU_XX.wav
sr=44100
1074_TIE_SAD_XX.wav
sr=44100
1074_TSI_ANG_XX.wav
sr=44100
1074_TSI_DIS_XX.wav
sr=44100
1074_TSI_FEA_XX.wav
sr=44100
1074_TSI_HAP_XX.wav
sr=44100
1074_TSI_NEU_XX.wav
sr=44100
1074_TSI_SAD_XX.wav
sr=44100
1074_WSI_ANG_XX.wav
sr=44100
1074_WSI_DIS_XX.wav
sr=44100
1074_WSI_FEA_XX.wav
sr=44100
1074_WSI_HAP_XX.wav
sr=44100
1074_WSI_NEU_XX.wav
sr=44100
1074_WSI_SAD_XX.wav
sr=44100
1075_DFA_ANG_XX.wav
sr=44100
1075_DFA_DIS_XX.wav
sr=44100
1075_

sr=44100
1079_IEO_ANG_HI.wav
sr=44100
1079_IEO_ANG_LO.wav
sr=44100
1079_IEO_ANG_MD.wav
sr=44100
1079_IEO_DIS_HI.wav
sr=44100
1079_IEO_DIS_LO.wav
sr=44100
1079_IEO_DIS_MD.wav
sr=44100
1079_IEO_FEA_HI.wav
sr=44100
1079_IEO_FEA_LO.wav
sr=44100
1079_IEO_FEA_MD.wav
sr=44100
1079_IEO_HAP_HI.wav
sr=44100
1079_IEO_HAP_LO.wav
sr=44100
1079_IEO_HAP_MD.wav
sr=44100
1079_IEO_NEU_XX.wav
sr=44100
1079_IEO_SAD_HI.wav
sr=44100
1079_IEO_SAD_LO.wav
sr=44100
1079_IEO_SAD_MD.wav
sr=44100
1079_IOM_ANG_XX.wav
sr=44100
1079_IOM_DIS_XX.wav
sr=44100
1079_IOM_FEA_XX.wav
sr=44100
1079_IOM_HAP_XX.wav
sr=44100
1079_IOM_NEU_XX.wav
sr=44100
1079_IOM_SAD_XX.wav
sr=44100
1079_ITH_ANG_XX.wav
sr=44100
1079_ITH_DIS_XX.wav
sr=44100
1079_ITH_FEA_XX.wav
sr=44100
1079_ITH_HAP_XX.wav
sr=44100
1079_ITH_NEU_XX.wav
sr=44100
1079_ITH_SAD_XX.wav
sr=44100
1079_ITS_ANG_XX.wav
sr=44100
1079_ITS_DIS_XX.wav
sr=44100
1079_ITS_FEA_XX.wav
sr=44100
1079_ITS_HAP_XX.wav
sr=44100
1079_ITS_NEU_XX.wav
sr=44100
1079_ITS_SAD_XX.wav
sr=44100
1079_

sr=44100
1089_IWL_HAP_XX.wav
sr=44100
1089_IWL_NEU_XX.wav
sr=44100
1089_IWL_SAD_XX.wav
sr=44100
1089_IWW_ANG_XX.wav
sr=44100
1089_IWW_DIS_XX.wav
sr=44100
1089_IWW_FEA_XX.wav
sr=44100
1089_IWW_HAP_XX.wav
sr=44100
1089_IWW_NEU_XX.wav
sr=44100
1089_IWW_SAD_XX.wav
sr=44100
1089_MTI_ANG_XX.wav
sr=44100
1089_MTI_DIS_XX.wav
sr=44100
1089_MTI_FEA_XX.wav
sr=44100
1089_MTI_HAP_XX.wav
sr=44100
1089_MTI_NEU_XX.wav
sr=44100
1089_MTI_SAD_XX.wav
sr=44100
1089_TAI_ANG_XX.wav
sr=44100
1089_TAI_DIS_XX.wav
sr=44100
1089_TAI_FEA_XX.wav
sr=44100
1089_TAI_HAP_XX.wav
sr=44100
1089_TAI_NEU_XX.wav
sr=44100
1089_TAI_SAD_XX.wav
sr=44100
1089_TIE_ANG_XX.wav
sr=44100
1089_TIE_DIS_XX.wav
sr=44100
1089_TIE_FEA_XX.wav
sr=44100
1089_TIE_HAP_XX.wav
sr=44100
1089_TIE_NEU_XX.wav
sr=44100
1089_TIE_SAD_XX.wav
sr=44100
1089_TSI_ANG_XX.wav
sr=44100
1089_TSI_DIS_XX.wav
sr=44100
1089_TSI_FEA_XX.wav
sr=44100
1089_TSI_HAP_XX.wav
sr=44100
1089_TSI_NEU_XX.wav
sr=44100
1089_TSI_SAD_XX.wav
sr=44100
1089_WSI_ANG_XX.wav
sr=44100
1089_

NameError: name 'image_folder_path' is not defined